<a href="https://colab.research.google.com/github/minhbao1705/pytorch_tutorial/blob/main/Autograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Trong phần này sẽ tìm hiểu về cách tính đạo hàm bằng cách sử dụng autograd package trong Pytorch

# **1 - The Autograd package**

Autograd package cung cấp một sự khác biệt tự động cho tất cả các hoạt động của tensor. Rất dễ dàng để sử dụng đạo hàm trong pytorch bằng cách chỉ cho nó biết rằng tensor cần được đạo hàm bằng requires_grad. Với việc thiết lập thuộc tính này, các phép toán trên tensor đều được theo dõi trên một đồ thị tính toán.

In [ ]:
import torch

x = torch.randn(3, requires_grad=True)
y = x + 2

In [ ]:
"""
  - ở cell trên y đã được tạo ra bởi kết quả của phép tính x + 2, vì vậy nó sẽ tạo một thuộc tính grad_fn.
  - grad_fn: tham chiếu đến một hàm đã tạo tensor
"""
print(x) # Đã được tạo ở cell trên -> grad_fn lúc này là None
print(y)
print(y.grad_fn)

tensor([ 1.7603, -1.2176,  0.3986], requires_grad=True)
tensor([3.7603, 0.7824, 2.3986], grad_fn=<AddBackward0>)


In [ ]:
# Thực hiện các phép tính khác trên y
z = y * 3
print(z)
z = z + 3
print(z)

tensor([11.2808,  2.3471,  7.1957], grad_fn=<MulBackward0>)
tensor([14.2808,  5.3471, 10.1957], grad_fn=<AddBackward0>)


# **2 - Tính đạo hàm với lan truyền ngược**

Khi hoàn tất quá trình tính toán, ta có thể gọi .backward() và tất cả giá trị đạo hàm sẽ được tính toán một cách tự động. Giá trị đạo hàm của những tensor này sẽ được tích lũy vào trong thuộc tính.grad. Nó chính là đạo hàm riêng của tensor.

In [ ]:
z = z.mean()
z.backward()
print(x.grad) # dz/dx

tensor([1., 1., 1.])


In [ ]:
x = torch.randn(5, requires_grad=True)
print(x)
y = x*2
for _ in range(10):
  y = y * 2

print(y)
print(y.shape)

tensor([-1.8002,  0.4505,  0.2386,  0.1756,  2.1282], requires_grad=True)
tensor([-3686.8625,   922.6259,   488.6931,   359.5541,  4358.5483],
       grad_fn=<MulBackward0>)
torch.Size([5])


In [ ]:
w = torch.tensor([0.1, 1.0, 0.0001, 0.01, 0.001], dtype=torch.float32)
y.backward(w)
print(x.grad)

tensor([2.0480e+02, 2.0480e+03, 2.0480e-01, 2.0480e+01, 2.0480e+00])


# **3 - Stop a tensor from tracking history**

Trong quá trình huấn luyện, khi chúng ta muốn cập nhật trọng số thì thao tác cập nhật này không nên là một phần của phép tính đạo hàm. Chúng ta có 3 sự lựa chọn cho việc dừng quá trình đạo hàm và cập nhật tham số như sau:
- x.requires_grad_false()
- x.detach()
- wrap in with torch.no_grad():

**.requires_grad_(...) thay đổi yêu cầu ngay tại vị trí cần yêu cầu đạo hàm**